In [1]:
import pandas as pd
import numpy as np
import os
import math
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import MaxPooling1D
from keras.layers import Attention
from keras.layers import AdditiveAttention
from keras.layers import Embedding
from keras.callbacks import LearningRateScheduler
from keras.layers import Input
from CustomAttentionLayer import attention
from Evaluation import Evaluation
from contextlib import redirect_stdout



2023-05-11 19:52:31.234507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 19:52:31.727211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 19:52:31.727239: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 19:52:31.789551: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-11 19:52:32.921962: W tensorflow/stream_executor/platform/de

In [2]:
def VanillaLSTM(input_shape,LEARNING_RATE):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(1)) 
    
    custom_optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE)
    model.compile(loss='mae' , optimizer=custom_optimizer)
    return model

In [3]:
def BiLSTMWithAttention(input_shape,LEARNING_RATE):
    model = Sequential()
    # embedding_layer = Embedding()
    # model.add(embedding_layer)
    model.add(Bidirectional(LSTM(64, input_shape =input_shape, return_sequences= True)))
    # model.add(attention(return_sequences= True))
    model.add(Flatten())
    model.add(Dense(1)) 
    custom_optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE)    
    model.compile(loss = 'mae', optimizer = custom_optimizer)   
    return model

In [4]:
def CNNGRU(input_shape, LEARNING_RATE):
    # Define model.
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape= input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(GRU(64, activation='relu', return_sequences=True))
    model.add(attention(return_sequences=True))
    model.add(Flatten())
    model.add(Dense(1))
    custom_optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE)
    model.compile(loss='mae', optimizer=custom_optimizer)
    
    return model

In [14]:
def CNNBiLSTM(input_shape,LEARNING_RATE):
        
    # Define model.
    model = Sequential()
    model.add(Input(shape = input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Bidirectional(LSTM(200, activation='relu',input_shape=input_shape, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(Bidirectional(LSTM(200,activation='relu', input_shape=input_shape, return_sequences = True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(200,activation='relu', input_shape=input_shape, return_sequences = True)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1 ))          
    custom_optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE)
    model.compile(loss='mae', optimizer=custom_optimizer)
    return model

In [6]:
def createDF(clustername):
    
    cluster = pd.read_csv(clustername, header=0,index_col=0)
    container_names = cluster['Series'].to_list()
    containers = []
    for name in container_names:
        df = pd.read_csv('../data/engineered_data/'+name+'.csv', header=0,index_col=0)
        containers.append(df)
    dataset = pd.concat(containers)
    dataset.sort_index(inplace=True)
    cluster_size = len(containers)
    return dataset, cluster_size

In [7]:
def removeOutlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df

In [8]:
def lagFeatures(future_steps, past_steps, data):
    input_features =[]
    target_features= []
    # Reformat input data into a shape: (n_samples x timesteps x n_features)
    # In my example, my df_for_training_scaled has a shape (12823, 5)
    # 12823 refers to the number of data points and 5 refers to the columns (multi-variables).
    for i in range(past_steps, len(data) - future_steps + 1):
        input_features.append(data[i - past_steps:i, :])
        target_features.append(data[i + future_steps - 1:i + future_steps, 0])
    return input_features, target_features

In [9]:
def decay_schedule(epoch, lr):
    # decay by 0.1 every 5 epochs; use `% 1` to decay after each epoch
    if (epoch % 3 == 0) and (epoch != 0):
        lr = lr * 0.9
    return lr

In [10]:

def inverseData(data, data_shape, scaler):
    ''' 
    inverting our data for evaluation
    '''
    data_copies = np.repeat(data,data_shape[1], axis = -1)
    inverse_data = scaler.inverse_transform(data_copies)[:,0]
    return inverse_data
    

In [29]:
cluster_name = '../data/clusters/Cluster 78.csv'
dataset , cluster_size = createDF(cluster_name)
dataset = removeOutlier(dataset)
# data variance for collecting variance
data_variance= dataset['cpu_util'].var()

values = dataset.values
# normalize features
# we fit and transform in differet lines for inversing transformation in evaluation section
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(values)
scaled = scaler.transform(values)
# Number of days we want to look into the future based on the past days.
future = 1
# Number of past days we want to use to predict the future.
if cluster_size >=70:
    past = 70
else:
    past = cluster_size

#producing lag features 
input_features , target_features = lagFeatures(future_steps=future, past_steps= past,data=scaled)
input_features, target_features = np.array(input_features), np.array(target_features)
#split train test sets
trainlength = int(input_features.shape[0]*0.7)
train_X = input_features[:trainlength, :, :]
test_X = input_features[trainlength:, :, :]
train_y = target_features[:trainlength, :]
test_y = target_features[trainlength:, :]
# prepare learning rate for dataset
lr = 10 ** (-len(str(input_features.shape[0]))-1)
lr_scheduler = LearningRateScheduler(decay_schedule)
####### model getting trained
model = BiLSTMWithAttention(input_shape =(train_X.shape[1], train_X.shape[2]), LEARNING_RATE=lr)
#####fit model
history = model.fit(train_X,
                    train_y,
                    epochs=150, 
                    batch_size=20,
                    validation_data=(test_X, test_y), 
                    verbose=2, 
                    shuffle=False,
                    callbacks=[lr_scheduler])
#####save model to file 
# if not os.path.exists(model_path):
#     os.makedirs(model_path)
# model_json = model.to_json()
# with open(model_path+'\\model.json', "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights(model_path+"\\model.h5")

test_predict = model.predict(test_X)
train_predict = model.predict(train_X)
shape = values.shape
inverse_train_predict = inverseData(train_predict.reshape(train_y.shape),shape, scaler)
inverse_train_y = inverseData(train_y,shape, scaler)
inverse_test_predict = inverseData(test_predict.reshape(test_y.shape),shape, scaler)
inverse_test_y = inverseData(test_y,shape, scaler)
train_metrics = Evaluation().get_evaluation(inverse_train_y,inverse_train_predict,'train')
test_metrics = Evaluation().get_evaluation(inverse_test_y,inverse_test_predict,'test')
model_metrics = pd.DataFrame({
    'cluster_name' : cluster_name,
    'cluster_size' : cluster_size,
    'data_variance' : data_variance,
    'test_mape' : test_metrics["mean_absolute_percentage_error"],
    'test_rmse' : test_metrics["root_mean_squared_error"],
    'train_mape':train_metrics["mean_absolute_percentage_error"],
    'train_rmse' : train_metrics["root_mean_squared_error"]
}, index = [0])


Epoch 1/150
128/128 - 8s - loss: 0.5197 - val_loss: 0.3494 - lr: 1.0000e-05 - 8s/epoch - 64ms/step
Epoch 2/150
128/128 - 3s - loss: 0.4940 - val_loss: 0.3267 - lr: 1.0000e-05 - 3s/epoch - 26ms/step
Epoch 3/150
128/128 - 3s - loss: 0.4566 - val_loss: 0.2910 - lr: 1.0000e-05 - 3s/epoch - 25ms/step
Epoch 4/150
128/128 - 3s - loss: 0.3969 - val_loss: 0.2349 - lr: 9.0000e-06 - 3s/epoch - 25ms/step
Epoch 5/150
128/128 - 3s - loss: 0.2974 - val_loss: 0.1466 - lr: 9.0000e-06 - 3s/epoch - 26ms/step
Epoch 6/150
128/128 - 3s - loss: 0.1628 - val_loss: 0.1080 - lr: 9.0000e-06 - 3s/epoch - 27ms/step
Epoch 7/150
128/128 - 3s - loss: 0.0957 - val_loss: 0.1190 - lr: 8.1000e-06 - 3s/epoch - 26ms/step
Epoch 8/150
128/128 - 3s - loss: 0.0870 - val_loss: 0.1220 - lr: 8.1000e-06 - 3s/epoch - 26ms/step
Epoch 9/150
128/128 - 3s - loss: 0.0857 - val_loss: 0.1203 - lr: 8.1000e-06 - 3s/epoch - 26ms/step
Epoch 10/150
128/128 - 3s - loss: 0.0849 - val_loss: 0.1185 - lr: 7.2900e-06 - 3s/epoch - 25ms/step
Epoch 11/

In [20]:
model_metrics


,cluster_name,cluster_size,data_variance,test_mape,test_rmse,train_mape,train_rmse
0,../data/clusters/Cluster 78.csv,28,4.598053,8.858519,1.466311,5.905414,1.114348


In [30]:
model_metrics

,cluster_name,cluster_size,data_variance,test_mape,test_rmse,train_mape,train_rmse
0,../data/clusters/Cluster 78.csv,28,4.598053,8.988439,1.485375,5.945541,1.1214


In [ ]:
model_metrics